# Исследование надежности заемщиков


**Описание проекта**

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель проекта**

Надо ответить на вопросы:
- Есть ли зависимость между количеством детей и возвратом кредита в срок? 
- Есть ли зависимость между семейным положением и возвратом кредита в срок? 
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок? 
- Как разные цели кредита влияют на его возврат в срок?

## Обзор данных

In [1]:
import pandas as pd

In [2]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [7]:
data['days_employed'] = data['days_employed'].abs()

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставлю их как есть. Тем более этот столбец не понадобится для исследования.

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [15]:
data['education'] = data['education'].str.lower()

In [16]:
data.duplicated().sum()

71

In [17]:
data = data.drop_duplicates()

### Категоризация данных

In [18]:
# Я использовал метод qcut(), чтобы категории по уровню дохода были примерно одинаковыми по количеству человек в них
# и создал новую колонку в data, где присвоил людям категории 'E', 'D', 'C', 'B', 'A'
bin_labels = ['E', 'D', 'C', 'B', 'A']
data['total_income_category'] = pd.qcut(data['total_income'], 5, labels=bin_labels) 
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,A
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,D
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,A
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,A
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,C
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,E
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,A


In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [20]:
def categorize_purpose(row): # создаю категории для целей кредита
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследование данных и ответы на вопросы

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
data_pivot_children = data.pivot_table( 
    index=['children'], columns='debt', values='education', aggfunc='count'
)
data_pivot_children = data_pivot_children.fillna(0)
data_pivot_children['отношение'] = data_pivot_children[1] / data_pivot_children[0] 
#Столбец "отношение" показывает отношение случаев просрочки платежа по кредиту к случаям когда просрочки не было
data_pivot_children

debt,0,1,отношение
children,,,
0,13028.0,1063.0,0.081593
1,4364.0,444.0,0.101742
2,1858.0,194.0,0.104413
3,303.0,27.0,0.089109
4,37.0,4.0,0.108108
5,9.0,0.0,0.000000


**Вывод:** Исходя из данных столбца "отношение" можно сделать вывод, что люди у которых нет детей, чаще других погашают кредит в срок, реже всех погашают кредит в срок люди с 2 детьми. Выборку с 3, 4 и 5 детьми мы не учитываем т. к. данных значений слишком мало.  

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
data_pivot_family = data.pivot_table( 
    index=['family_status'], columns='debt', values='education', aggfunc='count'
)
data_pivot_family['отношение'] = data_pivot_family[1] / data_pivot_family[0]
#Столбец "отношение" показывает отношение случаев просрочки платежа по кредиту к случаям когда просрочки не было
data_pivot_family

debt,0,1,отношение
family_status,,,
Не женат / не замужем,2523,273,0.108205
в разводе,1105,84,0.076018
вдовец / вдова,888,63,0.070946
гражданский брак,3749,385,0.102694
женат / замужем,11334,927,0.081789


**Вывод:** Исходя из данных столбца "отношение" можно сделать вывод, что вдовы/вдовцы, чаще других погашают кредит в срок, но так как данных значений меньше 5% то они могут быть нерепрезентативны, и в таком случае, чаще других погашают кредит в срок люди находящиеся в разводе, а реже всех те кто не женат/не замужем. 

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
data_pivot_income_qcut = data.pivot_table( 
    index=['total_income_category'], columns='debt', values='education', aggfunc='count'
)
data_pivot_income_qcut['отношение'] = data_pivot_income_qcut[1] / data_pivot_income_qcut[0]
#Столбец "отношение" показывает отношение случаев просрочки платежа по кредиту к случаям когда просрочки не было
data_pivot_income_qcut

debt,0,1,отношение
total_income_category,,,
E,3923,344,0.087688
D,3908,358,0.091607
C,3893,373,0.095813
B,3908,358,0.091607
A,3967,299,0.075372


**Вывод:** Исходя из данных столбца "отношение" можно сделать вывод, что люди, которым присвоена категория 'A', то есть те у кого доход 214604-2265604, намного чаще погашают кредит в срок, дальше в порядке убывания идут категории 'E', 'D', 'B', 'C'.

### Как разные цели кредита влияют на его возврат в срок?

In [25]:
data_pivot_purpose = data.pivot_table( 
    index=['purpose_category'], columns='debt', values='education', aggfunc='count'
)
data_pivot_purpose['отношение'] = data_pivot_purpose[1] / data_pivot_purpose[0]
#Столбец "отношение" показывает отношение случаев просрочки платежа по кредиту к случаям когда просрочки не было
data_pivot_purpose

debt,0,1,отношение
purpose_category,,,
операции с автомобилем,3879,400,0.103119
операции с недвижимостью,9971,780,0.078227
получение образования,3619,369,0.101962
проведение свадьбы,2130,183,0.085915


**Вывод:** Исходя из данных столбца "отношение" можно сделать вывод, что люди, которые берут кредит на операции с недвижимостью, погашают его в срок чаще остальных, а реже всех погашают кредит в срок, те кто берёт его на операции с автомобилем. 

## Общий вывод

Общий вывод: количество детей, семейное положение, уровень дохода и цель кредита влияют на погашение кредита в срок. А именно наиболее способными потенциальными заёмщиками для погашения кредита в срок являются те:
- у кого нет детей
- вдовы/вдовцы, люди находящиеся в разводе
- те у кого доход 214604-2265604 (категория 'A')
- и те кто берёт кредит на операции с недвижимостью.

Наименее же способными потенциальными заёмщиками для погашения кредита в срок являются те:
- у кого 2 ребёнка
- кто не женат/не замужем
- те у кого доход 132113-161380 (категория 'C')
- и те кто берёт кредит на операции с автомобилем.